# Motion Correction
This script will use "mcflirt" function of FSL to run estimate motion parameters.
<br>
Here we use the middle volume as reference. It also applies final transformations using spline interpolation.
<br>
The main outputs of this script are the realligned timeseries and the 6 estimated motion parameters (3 rotations + 3 traslations). These parmeters will be used to regress out the motion artificats from bold data.

-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import glob
import os
import subprocess

data_path = '/mnt/d/Data/Human/ObjectCategorization/analysis2'
bold_dirs = glob.glob('%s/sub*/ses-0*/func/*rest*.nii.gz'%(data_path))

for curr_bold in bold_dirs:
    exp_name = curr_bold.split('_')[-2].split('-')[-1]
    sub_dir = os.path.join(os.path.dirname(curr_bold),exp_name)    
    print('Currect direcotry is: \n\t%s'%(sub_dir))
    
    # change directory to the res_dir 
    print('\t 1) Changing the working directory to %s'%(sub_dir))
    os.chdir(sub_dir)
    
    #motion correction

    print('\t 2) Motion Correction using "MCFLIRT" function of FSL ...')
    os.system("mcflirt -in %s -out bold_mcf -mats -plots -rmsrel -rmsabs -spline_final"%(curr_bold))
    os.system("mkdir -p mc ; mv -f bold_mcf.mat bold_mcf.par bold_mcf_abs.rms bold_mcf_abs_mean.rms bold_mcf_rel.rms bold_mcf_rel_mean.rms  mc")
    os.system("fsl_tsplot -i %s/bold_mcf.par -t 'MCFLIRT estimated rotations (radians)' -u 1 --start=1 --finish=3 -a x,y,z -w 640 -h 144 -o %s/rot.png"%('mc','mc'))
    os.system("fsl_tsplot -i %s/bold_mcf.par -t 'MCFLIRT estimated translations (mm)' -u 1 --start=4 --finish=6 -a x,y,z -w 640 -h 144 -o %s/trans.png"%('mc','mc'))
    os.system("fsl_tsplot -i %s/bold_mcf_abs.rms,%s/bold_mcf_rel.rms -t 'MCFLIRT estimated mean displacement (mm)' -u 1 -w 640 -h 144 -a absolute,relative -o %s/disp.png"%('mc','mc','mc')) 
  

### plot estimated motion parameters


In [ ]:
%matplotlib inline

from IPython.display import display, Image
import matplotlib.pyplot as plt
from pathlib import Path
import glob

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find motion correction directories
mc_dirs = list(data_path.glob("sub*/ses-0*/func/rest*/mc"))

# Process each motion correction directory
for curr_mc in mc_dirs:
    sub, session,func,run = curr_mc.parts[-5:-1]  # Extract subject/session dynamically
    print(f"Processing: {sub}/{session}/{run}")

    # Ensure directory exists
    if not curr_mc.exists():
        print(f"Skipping {curr_mc}, directory does not exist!")
        continue

    # Display images
    plt.figure(figsize=(6, 4))  # Improve figure sizing
    display(Image(filename=str(curr_mc / "rot.png")))
    display(Image(filename=str(curr_mc / "trans.png")))